In [1]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import h5py
%matplotlib inline

sys.path.insert(0, os.environ["HOME"] + '/Programs/ana_cont')
from ana_cont import continuation as cont

In [2]:
f = h5py.File('/home/josef/Downloads/sigma_cont.h5','r')
tail_0 = f['current_sigma/site_0/tail_0/value'][()]
tail_1 = f['current_sigma/site_0/tail_1/value'][()]
niw = 20
n_orb = 4
siw = (f['current_sigma/site_0/values/value'][...,:niw] - tail_0[:,:,None])
print(tail_0.real)
print(tail_0.imag)
for i in range(n_orb):
    siw[i,i] /= tail_1[i,i]

OSError: Unable to open file (unable to open file: name = '/home/josef/Downloads/sigma_cont.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
beta = 20.
wn = np.pi/beta * (2.*np.arange(niw) + 1.)
#wgrid = np.linspace(-15., 15., num=1001, endpoint=True)
wgrid = 20. * np.tan(np.linspace(-np.pi/2.5, np.pi/2.5, num=1001, endpoint=True))/np.tan(np.pi/2.5)
n_orb = 4
model_diag = np.exp(-wgrid**2/(2.*8**2))
model_diag /= np.trapz(model_diag, wgrid)
errfac = 0.005
err = 0.002 + errfac * wn**2./wn[-1]**2

In [ ]:
sol_diag=[]
for i in range(n_orb):
    probl = cont.AnalyticContinuationProblem(im_axis=wn, re_axis=wgrid, im_data=siw[i,i], kernel_mode='freq_fermionic')
    sol_diag.append(
        probl.solve(method='maxent_svd', 
                    model=model_diag, 
                    stdev=err, 
                    alpha_determination='classic', 
                    offdiag=False, preblur=True, blur_width=0.15)[0])


In [ ]:
for i in range(n_orb):
    plt.plot(wgrid, sol_diag[i].A_opt)
plt.show()
fig,ax = plt.subplots(ncols=4, nrows=1, figsize=(20,4))
for i in range(n_orb):
    ax[i].errorbar(wn, siw[i,i].real, yerr=err)
    ax[i].plot(wn, sol_diag[i].backtransform.real, ls='--')
    ax[i].errorbar(wn, siw[i,i].imag, yerr=err)
    ax[i].plot(wn, sol_diag[i].backtransform.imag, ls='-')
plt.show()

In [ ]:
sol_offd = []
for i in range(n_orb):
    sol_offd.append([])
    for j in range(i):
        s = siw[i,j]
        if np.any(np.abs(s)>0.00001) and np.all(np.isfinite(s)):
            model = np.sqrt(sol_diag[i].A_opt * sol_diag[j].A_opt)
            probl = cont.AnalyticContinuationProblem(im_axis=wn, re_axis=wgrid, im_data=s, kernel_mode='freq_fermionic')
            sol_offd[i].append(
            probl.solve(method='maxent_svd', 
                        model=model, 
                        stdev=err, 
                        alpha_determination='classic', 
                        offdiag=True, preblur=True, blur_width=0.15)[0])
        else:
            sol_offd[i].append(None)

In [ ]:
fig, ax = plt.subplots(nrows=n_orb, ncols=n_orb, figsize=(20,20))

for i in range(n_orb):
    ax[i,i].plot(wgrid, sol_diag[i].A_opt)
    ax[i,i].set_xlim(-10.,10.)
    for j in range(i):
        if sol_offd[i][j] is not None:
            ax[i,j].plot(wgrid, sol_offd[i][j].A_opt)
            ax[i,j].set_xlim(-10.,10.)
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=n_orb, ncols=n_orb, figsize=(20,20))

for i in range(n_orb):
    ax[i,i].plot(wgrid, sol_diag[i].A_opt)
    ax[i,i].set_xlim(-2.,2.)
    for j in range(i):
        if sol_offd[i][j] is not None:
            ax[i,j].plot(wgrid, sol_offd[i][j].A_opt)
            ax[i,j].set_xlim(-2.,2.)
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=n_orb, ncols=n_orb, figsize=(20,20))

for i in range(n_orb):
    ax[i,i].errorbar(wn, siw[i,i].real, yerr=err)
    ax[i,i].plot(wn, sol_diag[i].backtransform.real, ls='--')
    ax[i,i].errorbar(wn, siw[i,i].imag, yerr=err)
    ax[i,i].plot(wn, sol_diag[i].backtransform.imag, ls='-')
    for j in range(i):
        if sol_offd[i][j] is not None:
            ax[i,j].errorbar(wn, siw[i,j].real, yerr=err)
            ax[i,j].plot(wn, sol_offd[i][j].backtransform.real, ls='--')
            ax[i,j].errorbar(wn, siw[i,j].imag, yerr=err)
            ax[i,j].plot(wn, sol_offd[i][j].backtransform.imag, ls='-')
plt.show()